In [1]:
import os
import re
import cloudpickle
import numpy as np
import IPython.display as ipd
from matplotlib import pyplot as plt

import torch

from ttsxai.interfaces.tts_interface import TTSInterface, get_text2mel, get_mel2wave

In [2]:
hop_length = 256
sampling_rate = 22050

In [3]:
with open('../data_analysis/analyze_phone_prosody/phone_data.pkl', "rb") as file:
    data_dict = cloudpickle.load(file)

In [4]:
text2mel = get_text2mel('tacotron2', 'cuda')
mel2wave = get_mel2wave('waveglow', 'cuda')

tts = TTSInterface(device='cuda', text2mel=text2mel, mel2wave=mel2wave ).to('cuda')

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



/nas/users/dahye/kw/tts/ttsxai/src/waveglow/glow.py:74: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2349.)
  W = torch.qr(torch.FloatTensor(c, c).normal_())[0]
/nas/users/dahye/kw/tts/ttsxai/src/tacotron2/stft.py:67: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = pad_center(fft_window, filter_length)


### Original case

In [94]:
from ttsxai.utils.utils import read_ljs_metadata
text = read_ljs_metadata(mode='train')['LJ001-0006']
text

'And it is worth mention in passing that, as an example of fine typography,'

In [95]:
tts.text2mel.set_modified_activations({})

# text = 'We have, therefore,'
tts_dict = tts.forward(text)
# text = tts_dict['text']
duration = tts_dict['duration']
phonesymbols = tts_dict['phonesymbols']
wave = tts_dict['wave']
activations = tts_dict['activations']

print(text)
ipd.Audio(wave, rate=sampling_rate)

torch.Size([1, 512, 67])
torch.Size([1, 512, 67])
torch.Size([1, 512, 67])
encoder_outputs.shape= torch.Size([1, 67, 512])
(80, 413)


/nas/users/dahye/kw/tts/ttsxai/src/tacotron2/audio_processing.py:50: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  win_sq = librosa_util.pad_center(win_sq, n_fft)


And it is worth mention in passing that, as an example of fine typography,


In [7]:
for i in range(len(phonesymbols)):
    print(phonesymbols[i], duration[i])

W 5
IY1 14
  0
HH 8
AE1 23
V 12
, 0
  0
DH 3
EH1 7
R 8
F 12
AO2 10
R 13
, 11


### replacement로 수정해보기
* 우선 lstm layer 만 수정하는 것을 가정

In [96]:
def activations2array(activations):
    if type(activations) == dict:
        return np.concatenate(list(activations.values()), axis=1)
    elif type(activations) == np.ndarray:
        return activations
    else:
        raise NotImplementedError

def activations2dict(activations, layer_shapes={'conv_0': 512, 'conv_1': 512, 'conv_2': 512, 'lstm': 512}):
    if type(activations) == dict:
        return activations
    
    if activations.shape[1] != sum(layer_shapes.values()):
        raise ValueError("Total size of layers does not match the array width.")
    
    activations_dict = {}
    current_index = 0
    for layer_name, shape in layer_shapes.items():
        activations_dict[layer_name] = activations[:, current_index:current_index + shape]
        current_index += shape
    
    return activations_dict


### max duration을 가지게 수정하기

In [97]:
# modified_activations = {
#     'lstm': activations['lstm'].copy()
# }

modified_activations_array = activations2array(activations).copy()

""" 
    We do not modify space/punctuation
"""

words = []
for w in re.split(r"([,;.\-\?\!\s+])", text.replace('.', '')):
    if w.lower() not in ['.', ',', '”','“', '"', "'", ';', '-', '(', ')', ':', '?', '!', ' ', '']:
        words.append(w.lower())

word_index_boundaries = []

s = 0 
for index, phone in enumerate(phonesymbols):
    if phone == " ":
        word_index_boundaries.append([s, e])
        s = index + 1
    elif phone not in ['.', ',', '”', '“', '"', "'", ';', '-', '(', ')', ':', '?', '!', ' ', '']:
        e = index
word_index_boundaries.append([s, e])

assert(len(words) == len(word_index_boundaries))

for idx, word_index_boundary in enumerate(word_index_boundaries):
    # 첫단어는 수정 X
    if idx == 0:
        continue

    s, e = word_index_boundary
    # if words[idx] == 'have':
    print(words[idx], s, e, phonesymbols[s:e+1], np.sum(duration[s:e+1]))
    for p_idx in range(s, e + 1):
        print(f'  {p_idx}', phonesymbols[p_idx])

        phone = phonesymbols[p_idx]
        phone_data = data_dict[phone]
        phone_duration_values = np.array(phone_data['duration'])
        phone_activations = np.array(phone_data['activations'])

        # filter invalid ...
        valid_ids = phone_duration_values > 0
        phone_duration_values = phone_duration_values[valid_ids]
        phone_activations = phone_activations[valid_ids]

        max_phone_duration_index = np.argmax(phone_duration_values)
        max_phone_duration_activation = phone_activations[max_phone_duration_index]
        
        # modify
        modified_activations_array[p_idx, :] = max_phone_duration_activation

modified_activations_dict = activations2dict(modified_activations_array)
# lstm 만 수정
del modified_activations_dict['conv_0']
del modified_activations_dict['conv_1']
del modified_activations_dict['conv_2']
modified_activations_dict['lstm'] = torch.from_numpy(modified_activations_dict['lstm'][None, :]).cuda()

tts.text2mel.set_modified_activations(modified_activations_dict)

new_tts_dict = tts.forward(text)
new_duration = new_tts_dict['duration']
new_phonesymbols = new_tts_dict['phonesymbols']
new_wave = new_tts_dict['wave']

for i in range(len(new_phonesymbols)):
    print(new_phonesymbols[i], new_duration[i])

print(text)
ipd.Audio(new_wave, rate=sampling_rate)

it 4 5 ['IH1', 'T'] 8
  4 IH1
  5 T
is 7 8 ['IH1', 'Z'] 14
  7 IH1
  8 Z
worth 10 12 ['W', 'ER1', 'TH'] 30
  10 W
  11 ER1
  12 TH
mention 14 19 ['M', 'EH1', 'N', 'SH', 'AH0', 'N'] 40
  14 M
  15 EH1
  16 N
  17 SH
  18 AH0
  19 N
in 21 22 ['IH0', 'N'] 10
  21 IH0
  22 N
passing 24 28 ['P', 'AE1', 'S', 'IH0', 'NG'] 45
  24 P
  25 AE1
  26 S
  27 IH0
  28 NG
that 30 32 ['DH', 'AE1', 'T'] 29
  30 DH
  31 AE1
  32 T
as 35 36 ['AE1', 'Z'] 12
  35 AE1
  36 Z
an 38 39 ['AE1', 'N'] 9
  38 AE1
  39 N
example 41 48 ['IH0', 'G', 'Z', 'AE1', 'M', 'P', 'AH0', 'L'] 50
  41 IH0
  42 G
  43 Z
  44 AE1
  45 M
  46 P
  47 AH0
  48 L
of 50 51 ['AH1', 'V'] 8
  50 AH1
  51 V
fine 53 55 ['F', 'AY1', 'N'] 29
  53 F
  54 AY1
  55 N
typography 57 65 ['T', 'AH0', 'P', 'AA1', 'G', 'R', 'AH0', 'F', 'IY0'] 68
  57 T
  58 AH0
  59 P
  60 AA1
  61 G
  62 R
  63 AH0
  64 F
  65 IY0
torch.Size([1, 512, 67])
torch.Size([1, 512, 67])
torch.Size([1, 512, 67])
### modify lstm
encoder_outputs.shape= torch.Size([1, 67, 512

### max duration을 가지게 수정하기 (단 같은 단어의 음소중에서)

In [99]:
# modified_activations = {
#     'lstm': activations['lstm'].copy()
# }

modified_activations_array = activations2array(activations).copy()

""" 
    We do not modify space/punctuation
"""

words = []
for w in re.split(r"([,;.\-\?\!\s+])", text.replace('.', '')):
    if w.lower() not in ['.', ',', '”','“', '"', "'", ';', '-', '(', ')', ':', '?', '!', ' ', '']:
        words.append(w.lower())

word_index_boundaries = []

s = 0 
for index, phone in enumerate(phonesymbols):
    if phone == " ":
        word_index_boundaries.append([s, e])
        s = index + 1
    elif phone not in ['.', ',', '”', '“', '"', "'", ';', '-', '(', ')', ':', '?', '!', ' ', '']:
        e = index
word_index_boundaries.append([s, e])

assert(len(words) == len(word_index_boundaries))

for idx, word_index_boundary in enumerate(word_index_boundaries):
    # 첫단어는 수정 X
    if idx == 0:
        continue
    
    s, e = word_index_boundary
    # if words[idx] == 'have':
    print(words[idx], s, e, phonesymbols[s:e+1], np.sum(duration[s:e+1]))
    for p_idx in range(s, e + 1):
        print(f'  {p_idx}', phonesymbols[p_idx])

        phone = phonesymbols[p_idx]
        phone_data = data_dict[phone]
        phone_duration_values = np.array(phone_data['duration'])
        phone_activations = np.array(phone_data['activations'])

        # filter different words
        phone_words = np.array(phone_data['word'])
        valid_ids = phone_words == words[idx]
        phone_duration_values = phone_duration_values[valid_ids]
        phone_activations = phone_activations[valid_ids]

        # filter invalid ...
        valid_ids = phone_duration_values > 0
        phone_duration_values = phone_duration_values[valid_ids]
        phone_activations = phone_activations[valid_ids]

        max_phone_duration_index = np.argmax(phone_duration_values)
        max_phone_duration_activation = phone_activations[max_phone_duration_index]
        
        # modify
        modified_activations_array[p_idx, :] = max_phone_duration_activation

modified_activations_dict = activations2dict(modified_activations_array)
# lstm 만 수정
del modified_activations_dict['conv_0']
del modified_activations_dict['conv_1']
del modified_activations_dict['conv_2']
modified_activations_dict['lstm'] = torch.from_numpy(modified_activations_dict['lstm'][None, :]).cuda()

tts.text2mel.set_modified_activations(modified_activations_dict)

new_tts_dict = tts.forward(text)
new_duration = new_tts_dict['duration']
new_phonesymbols = new_tts_dict['phonesymbols']
new_wave = new_tts_dict['wave']

for i in range(len(new_phonesymbols)):
    print(new_phonesymbols[i], new_duration[i])

print(text)
ipd.Audio(new_wave, rate=sampling_rate)

it 4 5 ['IH1', 'T'] 8
  4 IH1
  5 T
is 7 8 ['IH1', 'Z'] 14
  7 IH1
  8 Z
worth 10 12 ['W', 'ER1', 'TH'] 30
  10 W
  11 ER1
  12 TH
mention 14 19 ['M', 'EH1', 'N', 'SH', 'AH0', 'N'] 40
  14 M
  15 EH1
  16 N
  17 SH
  18 AH0
  19 N
in 21 22 ['IH0', 'N'] 10
  21 IH0
  22 N
passing 24 28 ['P', 'AE1', 'S', 'IH0', 'NG'] 45
  24 P
  25 AE1
  26 S
  27 IH0
  28 NG
that 30 32 ['DH', 'AE1', 'T'] 29
  30 DH
  31 AE1
  32 T
as 35 36 ['AE1', 'Z'] 12
  35 AE1
  36 Z
an 38 39 ['AE1', 'N'] 9
  38 AE1
  39 N
example 41 48 ['IH0', 'G', 'Z', 'AE1', 'M', 'P', 'AH0', 'L'] 50
  41 IH0
  42 G
  43 Z
  44 AE1
  45 M
  46 P
  47 AH0
  48 L
of 50 51 ['AH1', 'V'] 8
  50 AH1
  51 V
fine 53 55 ['F', 'AY1', 'N'] 29
  53 F
  54 AY1
  55 N
typography 57 65 ['T', 'AH0', 'P', 'AA1', 'G', 'R', 'AH0', 'F', 'IY0'] 68
  57 T
  58 AH0
  59 P
  60 AA1
  61 G
  62 R
  63 AH0
  64 F
  65 IY0
torch.Size([1, 512, 14])
torch.Size([1, 512, 14])
torch.Size([1, 512, 14])
### modify lstm
encoder_outputs.shape= torch.Size([1, 67, 512

/nas/users/dahye/kw/tts/ttsxai/src/tacotron2/audio_processing.py:50: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  win_sq = librosa_util.pad_center(win_sq, n_fft)


W 0
IY1 18
  6
HH 6
AE1 8
V 2
, 0
  13
DH 11
EH1 9
R 8
F 18
AO2 19
R 26
And it is worth mention in passing that, as an example of fine typography,


### top 5% 로 수정

In [100]:
# modified_activations = {
#     'lstm': activations['lstm'].copy()
# }

modified_activations_array = activations2array(activations).copy()

""" 
    We do not modify space/punctuation
"""

words = []
for w in re.split(r"([,;.\-\?\!\s+])", text.replace('.', '')):
    if w.lower() not in ['.', ',', '”','“', '"', "'", ';', '-', '(', ')', ':', '?', '!', ' ', '']:
        words.append(w.lower())

word_index_boundaries = []

s = 0 
for index, phone in enumerate(phonesymbols):
    if phone == " ":
        word_index_boundaries.append([s, e])
        s = index + 1
    elif phone not in ['.', ',', '”', '“', '"', "'", ';', '-', '(', ')', ':', '?', '!', ' ', '']:
        e = index
word_index_boundaries.append([s, e])

assert(len(words) == len(word_index_boundaries))

for idx, word_index_boundary in enumerate(word_index_boundaries):
    # 첫단어는 수정 X
    if idx == 0:
        continue

    s, e = word_index_boundary
    # if words[idx] == 'have':
    print(words[idx], s, e, phonesymbols[s:e+1], np.sum(duration[s:e+1]))
    for p_idx in range(s, e + 1):
        print(f'  {p_idx}', phonesymbols[p_idx])

        phone = phonesymbols[p_idx]
        phone_data = data_dict[phone]
        phone_duration_values = np.array(phone_data['duration'])
        phone_activations = np.array(phone_data['activations'])

        # filter invalid ...
        valid_ids = phone_duration_values > 0
        phone_duration_values = phone_duration_values[valid_ids]
        phone_activations = phone_activations[valid_ids]

        phone_duration_percentile_90 = np.percentile(phone_duration_values, 95)
        top_10_percent_indices = np.where(phone_duration_values >= phone_duration_percentile_90)[0]

        # Retrieve the activations for these indices
        top_10_percent_activations = phone_activations[top_10_percent_indices]

        # Find the index of the lowest pitch within the top 20% activations
        lowest_duration_in_top_10_index = np.argmin(phone_duration_values[top_10_percent_indices])

        # Retrieve the activation with the lowest duration in the top 20%
        selected_phone_activation = top_10_percent_activations[lowest_duration_in_top_10_index]
        
        # modify
        modified_activations_array[p_idx, :] = selected_phone_activation

modified_activations_dict = activations2dict(modified_activations_array)
# lstm 만 수정
del modified_activations_dict['conv_0']
del modified_activations_dict['conv_1']
del modified_activations_dict['conv_2']
modified_activations_dict['lstm'] = torch.from_numpy(modified_activations_dict['lstm'][None, :]).cuda()

tts.text2mel.set_modified_activations(modified_activations_dict)

new_tts_dict = tts.forward(text)
new_duration = new_tts_dict['duration']
new_phonesymbols = new_tts_dict['phonesymbols']
new_wave = new_tts_dict['wave']

for i in range(len(new_phonesymbols)):
    print(new_phonesymbols[i], new_duration[i])

print(text)
ipd.Audio(new_wave, rate=sampling_rate)

it 4 5 ['IH1', 'T'] 8
  4 IH1
  5 T
is 7 8 ['IH1', 'Z'] 14
  7 IH1
  8 Z
worth 10 12 ['W', 'ER1', 'TH'] 30
  10 W
  11 ER1
  12 TH
mention 14 19 ['M', 'EH1', 'N', 'SH', 'AH0', 'N'] 40
  14 M
  15 EH1
  16 N
  17 SH
  18 AH0
  19 N
in 21 22 ['IH0', 'N'] 10
  21 IH0
  22 N
passing 24 28 ['P', 'AE1', 'S', 'IH0', 'NG'] 45
  24 P
  25 AE1
  26 S
  27 IH0
  28 NG
that 30 32 ['DH', 'AE1', 'T'] 29
  30 DH
  31 AE1
  32 T
as 35 36 ['AE1', 'Z'] 12
  35 AE1
  36 Z
an 38 39 ['AE1', 'N'] 9
  38 AE1
  39 N
example 41 48 ['IH0', 'G', 'Z', 'AE1', 'M', 'P', 'AH0', 'L'] 50
  41 IH0
  42 G
  43 Z
  44 AE1
  45 M
  46 P
  47 AH0
  48 L
of 50 51 ['AH1', 'V'] 8
  50 AH1
  51 V
fine 53 55 ['F', 'AY1', 'N'] 29
  53 F
  54 AY1
  55 N
typography 57 65 ['T', 'AH0', 'P', 'AA1', 'G', 'R', 'AH0', 'F', 'IY0'] 68
  57 T
  58 AH0
  59 P
  60 AA1
  61 G
  62 R
  63 AH0
  64 F
  65 IY0
torch.Size([1, 512, 67])
torch.Size([1, 512, 67])
torch.Size([1, 512, 67])
### modify lstm
encoder_outputs.shape= torch.Size([1, 67, 512

/nas/users/dahye/kw/tts/ttsxai/src/tacotron2/audio_processing.py:50: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  win_sq = librosa_util.pad_center(win_sq, n_fft)


AH0 3
N 4
D 5
  7
IH1 9
T 7
  10
IH1 10
Z 7
  0
W 10
ER1 19
TH 4
  0
M 10
EH1 17
N 12
SH 3
AH0 0
N 0
  15
IH0 5
N 8
  1
P 10
AE1 0
S 15
IH0 9
NG 8
  0
DH 4
AE1 25
T 10
, 17
  21
AE1 28
Z 14
  13
AE1 25
N 11
  13
IH0 4
G 9
Z 0
AE1 23
M 3
P 19
AH0 18
L 14
  12
AH1 0
V 15
  17
F 11
AY1 31
N 9
  11
T 0
AH0 0
P 0
AA1 0
G 0
R 0
AH0 0
F 0
IY0 0
, 0
And it is worth mention in passing that, as an example of fine typography,


### top 5% 로 수정 (단 같은 단어에서)

In [116]:
# modified_activations = {
#     'lstm': activations['lstm'].copy()
# }

modified_activations_array = activations2array(activations).copy()

""" 
    We do not modify space/punctuation
"""

words = []
for w in re.split(r"([,;.\-\?\!\s+])", text.replace('.', '')):
    if w.lower() not in ['.', ',', '”','“', '"', "'", ';', '-', '(', ')', ':', '?', '!', ' ', '']:
        words.append(w.lower())

word_index_boundaries = []

s = 0 
for index, phone in enumerate(phonesymbols):
    if phone == " ":
        word_index_boundaries.append([s, e])
        s = index + 1
    elif phone not in ['.', ',', '”', '“', '"', "'", ';', '-', '(', ')', ':', '?', '!', ' ', '']:
        e = index
word_index_boundaries.append([s, e])

assert(len(words) == len(word_index_boundaries))

for idx, word_index_boundary in enumerate(word_index_boundaries):
    # 첫단어는 수정 X
    if idx == 0:
        continue

    s, e = word_index_boundary
    
    print(words[idx], s, e, phonesymbols[s:e+1], np.sum(duration[s:e+1]))
    for p_idx in range(s, e + 1):
        print(f'  {p_idx}', phonesymbols[p_idx])

        phone = phonesymbols[p_idx]
        phone_data = data_dict[phone]
        phone_duration_values = np.array(phone_data['duration'])
        phone_activations = np.array(phone_data['activations'])

        # filter different words
        phone_words = np.array(phone_data['word'])
        valid_ids = phone_words == words[idx]
        phone_duration_values = phone_duration_values[valid_ids]
        phone_activations = phone_activations[valid_ids]

        # filter invalid ...
        valid_ids = phone_duration_values > 0
        phone_duration_values = phone_duration_values[valid_ids]
        phone_activations = phone_activations[valid_ids]

        phone_duration_percentile_90 = np.percentile(phone_duration_values, 95)
        top_10_percent_indices = np.where(phone_duration_values >= phone_duration_percentile_90)[0]

        # Retrieve the activations for these indices
        top_10_percent_activations = phone_activations[top_10_percent_indices]

        # Find the index of the lowest pitch within the top 20% activations
        lowest_duration_in_top_10_index = np.argmin(phone_duration_values[top_10_percent_indices])

        # Retrieve the activation with the lowest duration in the top 20%
        selected_phone_activation = top_10_percent_activations[lowest_duration_in_top_10_index]
        
        # modify
        modified_activations_array[p_idx, :] = selected_phone_activation

modified_activations_dict = activations2dict(modified_activations_array)
# lstm 만 수정
del modified_activations_dict['conv_0']
del modified_activations_dict['conv_1']
del modified_activations_dict['conv_2']
modified_activations_dict['lstm'] = torch.from_numpy(modified_activations_dict['lstm'][None, :]).cuda()

tts.text2mel.set_modified_activations(modified_activations_dict)

new_tts_dict = tts.forward(text)
new_duration = new_tts_dict['duration']
new_phonesymbols = new_tts_dict['phonesymbols']
new_wave = new_tts_dict['wave']

for i in range(len(new_phonesymbols)):
    print(new_phonesymbols[i], new_duration[i])

print(text)
ipd.Audio(new_wave, rate=sampling_rate)

it 4 5 ['IH1', 'T'] 8
  4 IH1
  5 T
is 7 8 ['IH1', 'Z'] 14
  7 IH1
  8 Z
worth 10 12 ['W', 'ER1', 'TH'] 30
  10 W
  11 ER1
  12 TH
mention 14 19 ['M', 'EH1', 'N', 'SH', 'AH0', 'N'] 40
  14 M
  15 EH1
  16 N
  17 SH
  18 AH0
  19 N
in 21 22 ['IH0', 'N'] 10
  21 IH0
  22 N
passing 24 28 ['P', 'AE1', 'S', 'IH0', 'NG'] 45
  24 P
  25 AE1
  26 S
  27 IH0
  28 NG
that 30 32 ['DH', 'AE1', 'T'] 29
  30 DH
  31 AE1
  32 T
as 35 36 ['AE1', 'Z'] 12
  35 AE1
  36 Z
an 38 39 ['AE1', 'N'] 9
  38 AE1
  39 N
example 41 48 ['IH0', 'G', 'Z', 'AE1', 'M', 'P', 'AH0', 'L'] 50
  41 IH0
  42 G
  43 Z
  44 AE1
  45 M
  46 P
  47 AH0
  48 L
of 50 51 ['AH1', 'V'] 8
  50 AH1
  51 V
fine 53 55 ['F', 'AY1', 'N'] 29
  53 F
  54 AY1
  55 N
typography 57 65 ['T', 'AH0', 'P', 'AA1', 'G', 'R', 'AH0', 'F', 'IY0'] 68
  57 T
  58 AH0
  59 P
  60 AA1
  61 G
  62 R
  63 AH0
  64 F
  65 IY0
torch.Size([1, 512, 67])
torch.Size([1, 512, 67])
torch.Size([1, 512, 67])
### modify lstm
encoder_outputs.shape= torch.Size([1, 67, 512

/nas/users/dahye/kw/tts/ttsxai/src/tacotron2/audio_processing.py:50: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  win_sq = librosa_util.pad_center(win_sq, n_fft)
